# Декораторы II

In [ ]:
import functools

In [ ]:
def validate_user_id(func):
    @functools.wraps(func)
    def wrapped(*args, **kwargs):
        if 'user_id' not in kwargs or kwargs['user_id'] == 0:
            print('user_id is wrong')
            return

        print('user_id is correct')
        return func(*args, **kwargs)
    
    return wrapped


def log_access(func):
    @functools.wraps(func)
    def wrapped(*args, **kwargs):
        print('{} was called'.format(func.__name__))
        return func(*args, **kwargs)
    return wrapped


@log_access
@validate_user_id
def feedback(user_id, message):
    print('Feedback received')


data = {
    'message': 'Your app is awesome!',
    'user_id': 42
}

feedback(**data)

In [ ]:
def bold(func):
    def wrapped():
        return "<b>" + func() + "</b>"
    return wrapped


def italic(func):
    def wrapped():
        return "<i>" + func() + "</i>"
    return wrapped


@bold
@italic
def hello():
    return "hello world"


# hello = bold(italic(hello))


print(hello())

# Итераторы


In [ ]:
iterator = iter([1, 2, 3])

In [ ]:
print(next(iterator))

In [ ]:
class EvenIterator:
    def __init__(self, start, end):
        self.current = start
        self.end = end
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current > self.end:
            raise StopIteration

        self.current += 2
        return self.current - 2
    
    
for num in EvenIterator(1, 10):
    print(num)
        

In [ ]:
class IndexIterable:
    def __init__(self, obj):
        self.obj = obj
        
    def __getitem__(self, index):
        return self.obj[index]


for letter in IndexIterable('123'):
    print(letter)

    
# Какой еще объект может быть вместо строки?

In [ ]:
import collections



print(isinstance(EvenIterator(1, 10), collections.Iterable))
print(isinstance(IndexIterable('123'), collections.Iterable))


# Асинхронные итераторы

In [ ]:
import asyncio


class Ranger:
    def __init__(self, limit):
        self.limit = limit

    def __aiter__(self):
        return self

    async def __anext__(self):
        await asyncio.sleep(1)
        if self.limit == 0:
            raise StopAsyncIteration
            
        self.limit -= 1
        return self.limit + 1
       

async def main():
    async for val in Ranger(10):
        print(val)

    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

# Генераторы

### Что такое генераторы и зачем они нужны?

In [ ]:
def fibonacci(number):
    a = b = 1
    for _ in range(number):
        yield a
        a, b = b, a + b
        

# for num in fibonacci(100):
#     print(num)

In [ ]:
def list_generator(list_obj):
    for item in list_obj:
        yield item
        print('After yielding {}'.format(item))


generator = list_generator([1, 2, 3])

In [ ]:
# print(next(generator))

In [ ]:
def accumulator():
    total = 0
    while True:
        value = yield total
        if not value:
            break

        total += value
        

g = accumulator()

# next(g)

# print(g.send(1))
# print(g.send(2))
# print(g.send(3))

# next(g)

In [ ]:
def catcher():
    try:
        while True:
            yield 1

    except ValueError:
        while True:
            yield 2
    

g = catcher()

next(g)

g.throw(ValueError)

next(g)

# Асинхронные генераторы

In [ ]:
async def arange(start, stop):
    current = start
    while current <= stop - 1:
        yield current
        current += 1


async def main():
    async for num in arange(10, 20):
        print(num)


loop = asyncio.get_event_loop()
loop.run_until_complete(main())

# Списочные выражения

In [ ]:
import random


def get_random_list(length):
    random_list = []
    for _ in range(length):
        random_list.append(random.random())
    
    return random_list


print(get_random_list(10))

In [ ]:
import random


def get_random_list(length):
    return [random.random()] * length

print(get_random_list(10))

In [ ]:
import random


def get_random_list(length):
    return [random.random() for _ in range(length)]

print(get_random_list(10))

In [ ]:
[str(num) for num in range(10)]

In [ ]:
[str(num) for num in range(10) if num % 2]

In [ ]:
[str(num) if num % 2 else num for num in range(10)]

In [ ]:
{num % 10 for num in range(100)}

In [ ]:
{num: num % 3 for num in range(20)}

In [ ]:
dict((x, x ** 2) for x in range(4))

In [ ]:
(num ** 2 for num in range(10))

In [ ]:
[i * j for i in range(5) for j in range(5, 10)]

In [ ]:
[[x for x in range(5)] for _ in range(5)]

In [ ]:
[sorted(x) for x in [[2, 1], [4, 3], [0, 1]]]

In [ ]:
[print(x) for x in (1, 2, 3)]

In [ ]:
a = [x ** 2 for x in range(1, 5)]
b = [x ** 3 for x in range(1, 5)]

list(zip(a, b))

In [ ]:
list(zip(a * 2, b))

# Асинхронные comprehensions

In [ ]:
async def main():
    return [num async for num in arange(10, 20)]

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

# Контекстные менеджеры

In [ ]:
with open('access_log.log', 'a') as f:
    f.write('New Access\n')


In [ ]:
class open_file:
    def __init__(self, filename, mode):
        self.f = open(filename, mode)
    
    def __enter__(self):
        return self.f
    
    def __exit__(self, *args):
        self.f.close()
        
        
with open_file('test.log', 'a') as f:
    f.write('lalala\n')
    
with open_file('test.log', 'r') as f:
    print(f.readlines())

In [ ]:
class suppress_exception:
    def __init__(self, exc_type):
        self.exc_type = exc_type
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type == self.exc_type:
            print('Nothing happend.')
            return True
    
    
with suppress_exception(ZeroDivisionError) as f:
    print(type(f).__name__)
    big_number = 1 / 0


In [ ]:
import contextlib


@contextlib.contextmanager
def printer(num):
    print('Enter')
    
    yield num ** 3
    
    print('Exit')

    
with printer(2) as p:
    print(p)

## Напишите контекстный менеджер который считает и выводит время, проведенное внутри него

## Как добавить возможность получить текущее пройденное время?

# Асинхронные контекстные менеджеры

In [ ]:
import aiohttp
import asyncio


async def fetch(client):
    async with client.get('http://python.org') as resp:
        assert resp.status == 200
        return await resp.text()


async def main(loop):
    async with aiohttp.ClientSession(loop=loop) as client:
        html = await fetch(client)
        print(html[:96])

        
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))

# Дескрипторы

In [ ]:
class Descriptor:
    def __get__(self, obj, obj_type):
        print('get')
        
    def __set__(self, obj, value):
        print('set')


class Class:
    attr = Descriptor()
    

instance = Class()

instance.attr
instance.attr = 10

In [ ]:
class Value:
    def __init__(self):
        self.value = None
    
    @staticmethod
    def _prepare_value(value):
        return value * 10

    def __get__(self, obj, obj_type):
        return self.value
    
    def __set__(self, obj, value):
        self.value = self._prepare_value(value)
    
    
class Class:
    attr = Value()

    
instance = Class()
instance.attr = 10

print(instance.attr)
print(Class.attr)

In [ ]:
# Что происходит в момент доступа к атрибуту — instance.attr (Чтение)

def pseudo_read():
    if hasattr(instance.__class__, 'attr'):
        obj = instance.__class__.attr
        obj_type = obj.__class__

        if hasattr(obj_type, '__get__') and (
            hasattr(obj_type, '__set__') or 'attr' not in instance.__dict__
        ):
            return obj_type.__get__(obj, instance, instance.__class__)

    return instance.__dict__['attr']   


# 1. Data Descriptors
# 2. instance.__dict__
# 3. Non-Data Descriptors
# 4. class.__dict__
# 5. base classes


# instance.attr => type(instance).__dict__['attr'].__get__(instance, type(instance))
# Class.attr => Class.__dict__['attr'].__get__(None, Class)
    

In [ ]:
# Что происходит во время — instance.attr = 10 (Записи)

def pseudo_write():
    if hasattr(instance.__class__, 'attr'):
        obj = instance.__class__.attr
        obj_type = obj.__class__

        if hasattr(obj_type, '__set__'):
            obj_type.__set__(obj, instance, 10)
            return

    instance.__dict__['attr'] = 10


## Напишите дескриптор, который пишет в переданный ему во время инициализации файл все присваиваемые значения

### Функции — это дескрипторы

In [ ]:
class Class:
    def method(self):
        pass
    
    
obj = Class()    
print(obj.method)
print(Class.method)

In [ ]:
def summator(x, y):
    return x + y


lolwhat = summator.__get__(10)
lolwhat(21)

### @classmethod & @staticmethod & @property — это дескрпиторы

In [ ]:
class Property:
    def __init__(self, getter):
        self.getter = getter

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
    
        return self.getter(obj)
    
    
class Class:
    @property
    def x(self):
        return 'x'
    
    @Property
    def y(self):
        return 'y'
    
    def get_z(self):
        return 'z'
    
    z = Property(get_z)
    

obj = Class()

print(obj.x)
print(obj.y)
print(obj.z)

## Напишите реализацию @staticmethod или @classmethod

# \_\_slots\_\_ (и здесь не без дескрипторов)

In [ ]:
class Class:
    __slots__ = ['hello']
    
    def __init__(self):
        self.hello = 'test'

        
obj = Class()

# Мета-классы

### Классы — это объекты

In [ ]:
class A:
    ...


In [ ]:
a = A()

print(type(a))
print(type(A))

print(isinstance(a, A))
print(isinstance(A, type))

print(issubclass(A, type))
print(issubclass(A, object))

![](https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/instance-of.png)


In [ ]:
def dummy_factory():
    class Class:
        pass
    
    return Class


Dummy = dummy_factory()
obj = Dummy()

In [ ]:
class Meta(type):
    def __new__(cls, name, parents, attrs):
        print('Creating {}'.format(name))
        if 'class_id' not in attrs:
            attrs['class_id'] = name.lower()
        
         return super().__new__(cls, name, parents, attrs)
    

class A(metaclass=Meta):
    pass


A.class_id

In [ ]:
class Meta(type):
    def __init__(cls, name, bases, attrs):
        print('Initializing — {}'.format(name))
        if not hasattr(cls, 'registry'):
            cls.registry = {}
        else:
            cls.registry[name.lower()] = cls
            
        super().__init__(name, bases, attrs)
        
        
class Base(metaclass=Meta):
    pass


class A(Base):
    pass


class B(Base):
    pass


Base.registry

In [ ]:
from abc import ABC, abstractmethod


class Abstract(ABC):
    @abstractmethod
    def abstract_method(self):
        pass


Abstract()

# Несколько полезных модулей стандартной библиотеки

### itertools

In [ ]:
import itertools

In [ ]:
# list(itertools.chain(range(10), range(10, 20)))

In [ ]:
# list(itertools.combinations(range(5), 3))

In [ ]:
# list(itertools.permutations(range(3), 3))

In [ ]:
# g = itertools.cycle('ABC')
# for _ in range(10):
#     print(next(g))

In [ ]:
for number in itertools.count():
    if number > 10:
        break
    
#    print(number)

In [ ]:
# list(itertools.repeat(1, 10))

In [ ]:
# list(itertools.product('ABC', 'xy'))

In [ ]:
# list(itertools.zip_longest(range(10), range(5), fillvalue=None))

### re

In [ ]:
import re

In [ ]:
# sentence = 'Call me ASAP!!!11 8-990-1122239 srsly.'
# re.search(r' ([\d-]+) ', sentence).groups()

In [ ]:
# re.findall(r"(\w+)", "Python is the capital of Great Britain")

In [ ]:
# re.search(r"(123)", "a123zzb").group(1)

In [ ]:
# re.sub('(\.)', ' :)', 'Hello...')

In [ ]:
# ints = re.compile(r"(\d+)")
# re.findall(ints, "a123zaa223zb")

### collections

In [ ]:
import collections

In [ ]:
colors = ['red', 'blue', 'red', 'green', 'blue', 'blue']
counter = collections.Counter(colors)

# print(counter)

In [ ]:
# print(counter.most_common(2))

In [ ]:
d = collections.defaultdict(lambda: 'not here')   
# print(d['missing_key'])

In [ ]:
# collections.OrderedDict()

In [ ]:
class SuperList(collections.abc.MutableSequence):
    pass


### datetime

In [ ]:
from datetime import datetime, timedelta

In [ ]:
midterm = datetime.now() + timedelta(days=7)
# print(midterm.day)

In [ ]:
# print(datetime.now().strftime('%Y / %m / %d'))

In [ ]:
# print(datetime.strptime('2016-10-25', '%Y-%m-%d'))